In [39]:
import re

regex = re.compile('\d+')

regex.findall('#1 @ 1,39: 4x4') == ['1', '1', '39', '4', '4']

['1', '1', '39', '4', '4']

In [41]:
def bbox(code):
    regex = re.compile('\d+')
    nums = [int(el) for el in regex.findall(code)]
    cmin, rmin = nums[1], nums[2]
    cmax, rmax = cmin + nums[3], rmin + nums[4]
    return [slice(cmin, cmax), slice(rmin, rmax)]
    
assert bbox('#1 @ 1,3: 4x4') == [slice(1, 5), slice(3, 7)]
assert bbox('#1 @ 387,801: 11x22') == [slice(387, 387+11), slice(801, 801+22)]

In [42]:
file_name = '../data/day3.txt'
# file_name = '../data/day3_test.txt'

with open(file_name, 'r') as f:
    data = [bbox(l.rstrip()) for l in f.readlines()]

In [44]:
import numpy as np

cmax = max([d[0].stop for d in data])
rmax = max([d[1].stop for d in data])
m = np.zeros((rmax + 1, cmax + 1))


for d in data:
    m[d[0], d[1]] += 1
    
# sum(sum(m > 1)) == 4
sum(sum(m > 1))

115304

In [64]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
        
    Source
    -------
    https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation
    """
    bb1 = {'x1': bb1[0].start,
           'x2': bb1[0].stop,
           'y1': bb1[1].start,
           'y2': bb1[1].stop}
    bb2 = {'x1': bb2[0].start,
           'x2': bb2[0].stop,
           'y1': bb2[1].start,
           'y2': bb2[1].stop}
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

# with IOU check if bounding boxes overlap
# function above is copied from stack over flow
# and adjusted to work with the slices

r1 = bbox('#1 @ 1,3: 4x4')
r2 = bbox('#2 @ 3,1: 4x4')
r3 = bbox('#3 @ 5,5: 2x2')

assert get_iou(r1, r2) > 0
assert get_iou(r1, r3) == 0.0
assert get_iou(r3, r2) == 0.0


ids = set(np.arange(len(data)))
for i, r1 in enumerate(data):
    for j, r2 in enumerate(data[i+1:]):
        if get_iou(r1, r2) > 0:
            if i in ids:
                ids.remove(i)
            if i + j + 1 in ids:
                ids.remove(i + j + 1)
                
# answer is the number in the set plus one!
ids

{274}